In [1]:
import numpy as np
import pandas as pd
import cv2
import os
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df_train = pd.read_csv('/content/drive/MyDrive/Data/train.csv')
df_test = pd.read_csv('/content/drive/MyDrive/Data/test.csv')

In [4]:
df_train.head()

,id,filename,distance
0,0,0.jpg,83110
1,1,1.jpg,1035
2,2,2.jpg,20756
3,3,3.jpg,13286
4,4,4.jpg,13924


In [5]:
df_test.head()

,id,filename
0,16019,16019.jpg
1,16020,16020.jpg
2,16021,16021.jpg
3,16022,16022.jpg
4,16023,16023.jpg


In [6]:
import os

# Define the base path to your Google Drive
base_path = '/content/drive/MyDrive/tsp-cv'

def calculate_histograms(df):
  histograms = []
  count = 0
  # Join paths
  for x in df['filename']:
    count+=1
    print(count)
    file_path = os.path.join(base_path, x)

    image = cv2.imread(file_path)

    # Check if the image was successfully loaded
    if image is None:
        raise ValueError("Image not found or the path is incorrect")
    blue_channel = image[:, :, 0]
    hist = cv2.calcHist([blue_channel], [0], None, [256], [0, 256])
    histograms.append(hist.reshape(256,))

  # Convert the list of histograms to a NumPy array
  df['Histograms'] = histograms

In [8]:
calculate_histograms(df_train)

Streaming output truncated to the last 5000 lines.
11019
11020
11021
11022
11023
11024
11025
11026
11027
11028
11029
11030
11031
11032
11033
11034
11035
11036
11037
11038
11039
11040
11041
11042
11043
11044
11045
11046
11047
11048
11049
11050
11051
11052
11053
11054
11055
11056
11057
11058
11059
11060
11061
11062
11063
11064
11065
11066
11067
11068
11069
11070
11071
11072
11073
11074
11075
11076
11077
11078
11079
11080
11081
11082
11083
11084
11085
11086
11087
11088
11089
11090
11091
11092
11093
11094
11095
11096
11097
11098
11099
11100
11101
11102
11103
11104
11105
11106
11107
11108
11109
11110
11111
11112
11113
11114
11115
11116
11117
11118
11119
11120
11121
11122
11123
11124
11125
11126
11127
11128
11129
11130
11131
11132
11133
11134
11135
11136
11137
11138
11139
11140
11141
11142
11143
11144
11145
11146
11147
11148
11149
11150
11151
11152
11153
11154
11155
11156
11157
11158
11159
11160
11161
11162
11163
11164
11165
11166
11167
11168
11169
11170
11171
11172
11173
11174
11175
11176
1

In [9]:
df_train.head()

,id,filename,distance,Histograms
0,0,0.jpg,83110,"[67030.0, 3472.0, 13276.0, 3292.0, 7656.0, 561..."
1,1,1.jpg,1035,"[181272.0, 793.0, 4145.0, 577.0, 1202.0, 671.0..."
2,2,2.jpg,20756,"[452282.0, 13526.0, 62301.0, 11493.0, 24708.0,..."
3,3,3.jpg,13286,"[336589.0, 9267.0, 43919.0, 7853.0, 17532.0, 8..."
4,4,4.jpg,13924,"[311752.0, 8794.0, 43302.0, 7515.0, 18787.0, 9..."


In [11]:
df_train.to_csv('/content/drive/MyDrive/tsp-cv/train_histograms.csv', index=False)


# Step 3: Download the CSV file
from google.colab import files
files.download('/content/drive/MyDrive/tsp-cv/train_histograms.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [12]:
df_train = df_train[['Histograms', 'distance']]

In [22]:
from sklearn.model_selection import train_test_split

X = df_train.drop('distance', axis=1)
y = df_train['distance']

# Split the dataset
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Output the sizes of the splits
print(f"Training set size: {X_train.shape[0]} samples")
print(f"Validation set size: {X_val.shape[0]} samples")

X_train = np.array(X_train['Histograms'].tolist())
X_val = np.array(X_val['Histograms'].tolist())
y_train = np.array(y_train)
y_val = np.array(y_val)

Training set size: 12814 samples
Validation set size: 3204 samples


In [35]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam

model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
#model.add(Dropout(0.2))
#model.add(Dense(32, activation='relu'))
#model.add(Dropout(0.2))
model.add(Dense(16, activation='relu'))
model.add(Dense(1))

optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-7)

model.compile(loss = 'mean_squared_error', optimizer=optimizer, metrics=[RootMeanSquaredError(name="rmse")])

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-12, patience=60, verbose=1, mode='auto', restore_best_weights=True)
trained_model = model.fit(X_train, y_train, epochs=1000, steps_per_epoch=32, batch_size=256, validation_data = (X_val, y_val), callbacks=[monitor], verbose = 1)

Epoch 1/1000


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 699968320.0000 - rmse: 26346.5410 - val_loss: 223534176.0000 - val_rmse: 14951.0596
Epoch 2/1000
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 178996192.0000 - rmse: 13361.1797 - val_loss: 112548096.0000 - val_rmse: 10608.8691
Epoch 3/1000


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 97573376.0000 - rmse: 9863.0010 - val_loss: 62102600.0000 - val_rmse: 7880.5205
Epoch 4/1000
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 65260844.0000 - rmse: 8073.2798 - val_loss: 53566944.0000 - val_rmse: 7318.9443
Epoch 5/1000
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 52259244.0000 - rmse: 7226.9297 - val_loss: 40322380.0000 - val_rmse: 6349.9907
Epoch 6/1000
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 43014912.0000 - rmse: 6557.4365 - val_loss: 34470232.0000 - val_rmse: 5871.1353
Epoch 7/1000
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 35781988.0000 - rmse: 5975.1289 - val_loss: 25464050.0000 - val_rmse: 5046.1914
Epoch 8/1000
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 34660980.0000 - rmse: 5867.5825 - val_loss: 21804150.0000 - val_rmse: 4669.4912
Epoch 9/1000
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 22005636.0000 - rmse: 4688.2051 - val_loss: 18010992.0000 - val_rmse: 4243.9360
Epoch 10/1000
32/32 ━━━━━━━━━━━━━

In [24]:
calculate_histograms(df_test)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [26]:
df_test.to_csv('/content/drive/MyDrive/tsp-cv/test_histograms.csv', index=False)


# Step 3: Download the CSV file
from google.colab import files
files.download('/content/drive/MyDrive/tsp-cv/test_histograms.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [29]:
X_test = df_test['Histograms']
X_test = np.array(X_test.tolist())



126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [36]:
y_test = model.predict(X_test)

126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [37]:
df_test['distance'] = y_test
#df_test.drop('Histograms', axis=1, inplace=True)
df_test.to_csv('/content/drive/MyDrive/tsp-cv/predictions3.csv', index=False)


# Step 3: Download the CSV file
from google.colab import files
files.download('/content/drive/MyDrive/tsp-cv/predictions3.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>